# Les décorateurs


Dans ce chapitre, nous allons apprendre à utiliser et à créer des décorateurs.

Un **décorateur** est simplement une fonction qui modifie le comportement d'autres fonctions. C'est très utile lorsque l'on veut ajouter du même code à plusieurs fonctions existantes.

## Créer un décorateur `python`

Un décorateur permet de modifier le comportement d'une fonction. Il commence par un @ suivi de lettres ou de chiffres. Il se place sur la ligne précédant la définition d'une fonction. Comme ceci :

```python
@decorator
def fonction():
    """documentation de la fonction"""
    print("Great!")
```

`Python` intègre de nombreux décorateurs standards mais vous pouvez également en définir vous-même. Pourquoi ? Car un décorateur est simplement une fonction qui prend en paramètre une fonction et renvoie une (autre) fonction.

Voici un premier exemple de décoration de fonction.

In [1]:
def mon_decorateur(in_function):
    def out_function():
        """titi"""
        print("On peut faire des choses avant")
        print("On exécute maintenant la fonction")
        in_function()
        print("Et on peut faire des trucs après")
    
    out_function.__name__ = in_function.__name__ #La fonction décorée ne change pas de nom
    out_function.__doc__ = in_function.__doc__ #La doc est la même que la fct de base
    return out_function

In [2]:
def ma_fonction():
    print("Je ne fais rien du tout !")

ma_fonction()
print(ma_fonction.__name__)

ma_nouvelle_fonction = mon_decorateur(ma_fonction)

ma_nouvelle_fonction()
print(ma_nouvelle_fonction.__name__)

Je ne fais rien du tout !
ma_fonction
On peut faire des choses avant
On exécute maintenant la fonction
Je ne fais rien du tout !
Et on peut faire des trucs après
ma_fonction


In [3]:
@mon_decorateur
def ma_fonction2():
    print("Je ne fais rien du tout à nouveau!")
    
ma_fonction2()

On peut faire des choses avant
On exécute maintenant la fonction
Je ne fais rien du tout à nouveau!
Et on peut faire des trucs après


In [4]:
@mon_decorateur
def ma_fonction(x):
    """toto"""
    print("Je ne fais rien du tout !")
    print(x)
    
ma_fonction(0)
#print(ma_fonction.__name__)
#help(ma_fonction)

TypeError: out_function() takes 0 positional arguments but 1 was given

In [5]:
x = 2

def f(y):
    return x+y

f(0)

2

Lorsque la fonction a des arguments et retourne quelque chose, il est important de ne pas changer son comportement. 

Pour les arguments passés à la fonction, nous utilisons les mots clés `*args` et `**kwargs` que l'on a déjà vus dans le cours sur les fonctions. Cela permet de passer des nombres arbitraires d'arguments et d'arguments optionnels.

Pour le return, il suffit que la nouvelle fonction retourne le même résultat.

Voici un nouvel exemple.

In [6]:
def mon_decorateur_avec_arguments(in_function):
    def out_function(*args, **kwargs):
        print(f"La fonction s'appelle {in_function.__name__}")
        print("ses arguments sont ", args)
        print("ses arguments optionnels sont ", kwargs)
        out = in_function(*args, **kwargs)
        print(f"la fonction a été exécutée et a retourné {out}")
        return out
    
    out_function.__name__ = in_function.__name__
    out_function.__doc__ = in_function.__doc__
    
    return out_function

In [13]:
@mon_decorateur_avec_arguments
def f(x):
    return x*x

f(1)

La fonction s'appelle f
ses arguments sont  (1,)
ses arguments optionnels sont  {}
la fonction a été exécutée et a retourné 1


1

In [15]:
def decorateur_multiplie(f_in):
    def f_out(*args, **kwargs):
        # On multiplie tous les arguments d'entrée 2
        args_new = tuple([2*argk for argk in args])
        print(f"Argument(s) de départ = {args}")
        print(f"Argument(s) modifié(s) = {args_new}")
        out = f_in(*args_new, **kwargs)
        if not isinstance(out, tuple):
            out = (out, )
        out_new = tuple([outk/2 for outk in out])
        print(f"Sortie de départ = {out}")
        print(f"Sortie modifiée = {out_new}")
        if len(out_new) == 1:
            out_new = out_new[0]
        return out_new
    return f_out

@decorateur_multiplie
def f(x):
    return x*x

def g(x):
    return (2*x)**2/2

x = 7
# Avec le décorateur on a multiplié le résultat par 2 :
print(f"x*x = {x*x}")
print(f"f(x) = {f(x)}, g(x) = {g(x)}")

x*x = 49
Argument(s) de départ = (7,)
Argument(s) modifié(s) = (14,)
Sortie de départ = (196,)
Sortie modifiée = (98.0,)
f(x) = 98.0, g(x) = 98.0


In [16]:
@mon_decorateur_avec_arguments
def f(x, p=2):
    return x**p

f(2)
f(2, 3)
f(2, p=3)

La fonction s'appelle f
ses arguments sont  (2,)
ses arguments optionnels sont  {}
la fonction a été exécutée et a retourné 4
La fonction s'appelle f
ses arguments sont  (2, 3)
ses arguments optionnels sont  {}
la fonction a été exécutée et a retourné 8
La fonction s'appelle f
ses arguments sont  (2,)
ses arguments optionnels sont  {'p': 3}
la fonction a été exécutée et a retourné 8


8

Prenons un autre exemple concret que certains d'entre vous connaissent déjà (attention à l'abus de caféine).

In [17]:
class CoffeeMachine():
    """
    une classe pour la machine à café
    """
    water_level = 100  # variable partagée par toutes les instances
    
    def __init__(self):
        """initialise l'instance"""
        self.__nb_coffee = 0
    
    def _start_machine(self):
        """démarre la machine"""
        if self.water_level > 20:
            return True
        else:
            print("Ajoutez de l'eau s'il vous plait !")
            return False
          
    def __boil_water(self):
        """chauffe l'eau"""
        print("préchauffage...")
        
    def __grind_beans(self):
        """préparation du grain"""
        print("préparation du grain...")
        
    def __brew_coffee(self):
        """extraction du café"""
        print("extraction")
        
    def make_coffee(self):
        """prépare un nouveau café"""
        if self._start_machine():
            self.water_level -= 20
            self.__boil_water()
            self.__grind_beans()
            self.__brew_coffee()
            self.__nb_coffee += 1
            print("Prenez votre café !")

In [18]:
machine = CoffeeMachine()
for i in range(0, 5):
    machine.make_coffee()

préchauffage...
préparation du grain...
extraction
Prenez votre café !
préchauffage...
préparation du grain...
extraction
Prenez votre café !
préchauffage...
préparation du grain...
extraction
Prenez votre café !
préchauffage...
préparation du grain...
extraction
Prenez votre café !
Ajoutez de l'eau s'il vous plait !


Afin de débugger notre code, nous pourrions avoir envie que chaque fonction affiche son nom et les arguments qui lui sont passés.

In [20]:
debug = True

# Name est mon décorateur
def name(func):
    def inner(*args, **kwargs):
        if debug :
            print(f" running {func.__name__} {args} ".center(80, '-'))
        return func(*args, **kwargs)
    return inner

class CoffeeMachine():
    """
    une classe pour la machine à café
    """
    water_level = 100  # variable partagée par toutes les instances
    
    def __init__(self):
        """initialise l'instance"""
        self.__nb_coffee = 0
        self.nom = "ma machine"
        
    def __repr__(self):
        return self.nom
    
    @name
    def _start_machine(self):
        """démarre la machine"""
        if self.water_level > 20:
            return True
        else:
            print("Ajoutez de l'eau s'il vous plait !")
            return False
          
    @name
    def __boil_water(self):
        """chauffe l'eau"""
        print("préchauffage...")
        
    @name
    def __grind_beans(self):
        """préparation du grain"""
        print("préparation du grain...")
        
    @name
    def __brew_coffee(self):
        """extraction du café"""
        print("extraction")
        
    @name
    def make_coffee(self):
        """prépare un nouveau café"""
        if self._start_machine():
            self.water_level -= 20
            self.__boil_water()
            self.__grind_beans()
            self.__brew_coffee()
            self.__nb_coffee += 1
            print("Prenez votre café !")

In [21]:
machine = CoffeeMachine()
for i in range(0, 5):
    machine.make_coffee()

---------------------- running make_coffee (ma machine,) -----------------------
--------------------- running _start_machine (ma machine,) ---------------------
---------------------- running __boil_water (ma machine,) ----------------------
préchauffage...
--------------------- running __grind_beans (ma machine,) ----------------------
préparation du grain...
--------------------- running __brew_coffee (ma machine,) ----------------------
extraction
Prenez votre café !
---------------------- running make_coffee (ma machine,) -----------------------
--------------------- running _start_machine (ma machine,) ---------------------
---------------------- running __boil_water (ma machine,) ----------------------
préchauffage...
--------------------- running __grind_beans (ma machine,) ----------------------
préparation du grain...
--------------------- running __brew_coffee (ma machine,) ----------------------
extraction
Prenez votre café !
---------------------- running make_coffee (ma m

In [22]:
debug = False
machine = CoffeeMachine()
for i in range(0, 5):
    machine.make_coffee()

préchauffage...
préparation du grain...
extraction
Prenez votre café !
préchauffage...
préparation du grain...
extraction
Prenez votre café !
préchauffage...
préparation du grain...
extraction
Prenez votre café !
préchauffage...
préparation du grain...
extraction
Prenez votre café !
Ajoutez de l'eau s'il vous plait !


## Utiliser un ou plusieurs décorateurs `python`

De nombreux décorateurs ont déjà été codés et peuvent être utilisés. Vous pourrez les trouver à cette adresse [liste des décorateurs](https://wiki.python.org/moin/PythonDecoratorLibrary)

Un décorateur que nous utilisons souvent est le décorateur `@property` qui fonctionne seulement si la classe hérite de la classe `object` (à partir de python 3).


In [24]:
def _CtoK(C):
    return C + 273.15

def _FtoK(F):
    return (F+459.67)*5/9

def _KtoC(K):
    return K - 273.15

def _KtoF(K):
    return 9/5*K-459.67


class temperature(object):
    """une classe pour la température"""
    def __init__(self, K=0, C=None, F=None):
        self._K = K
        if K is not None:
            self._K = K
        if C is not None:
            self._K = _CtoK(C)
        if F is not None:
            self._K = _FtoK(F)

    def __str__(self):
        return f"La température vaut K={self._K:.2f} (C={_KtoC(self._K):.2f}, F={_KtoF(self._K):.2f})"

In [26]:
t = temperature(K=273)
t2 = temperature()
print(f" La température vaut, en Kelvin : {t._K}")
print(f" La température 2 vaut, en Kelvin : {t2._K}")
t = temperature(C=0)
print(f" A 0°C, la température vaut, en Kelvin : {t._K}")
t = temperature(F=10)
print(f" A 10°F, la température vaut, en Kelvin : {t._K}")
t = temperature(K=0,C=0)
print(f" La température vaut, en Kelvin : {t._K}")
t = temperature(C=0,F=10)
print(f" La température vaut, en Kelvin : {t._K}")
t = temperature(C=0,F=10)
t._K = -50
print(f" La température vaut, en Kelvin : {t._K}")

 La température vaut, en Kelvin : 273
 La température 2 vaut, en Kelvin : 0
 A 0°C, la température vaut, en Kelvin : 273.15
 A 10°F, la température vaut, en Kelvin : 260.92777777777775
 La température vaut, en Kelvin : 273.15
 La température vaut, en Kelvin : 260.92777777777775
 La température vaut, en Kelvin : -50


Maintenant on va empêcher de modifier la température en Kelvin manuellement à l'aide du décorateur @poperty :

In [41]:
class temperature(object):
    """une classe pour la température"""
    def __init__(self, K=0, C=None, F=None):
        self._K = K
        if K is not None:
            self._K = K
        if C is not None:
            self._K = _CtoK(C)
        if F is not None:
            self._K = _FtoK(F)

    @property
    def Kelvin(self):
        return self._K
    
    def __str__(self):
        return f"La température vaut K={self._K:.2f} (C={_KtoC(self._K):.2f}, F={_KtoF(self._K):.2f})"

Maintenant, ma classe se comporte comme si elle avait un attribut Kelvin toujours égal à _K. Mais il n'est **pas possible de le modifier**.

In [42]:
t = temperature(C=0)
# Ma classe se comporte comme si elle avait un attribut Kelvin toujours égal à _K
print(f" La température vaut, en Kelvin : {t.Kelvin}")
t = temperature(C=0)
t.Kelvin = -50
print(f" La température vaut, en Kelvin : {t.Kelvin}")

 La température vaut, en Kelvin : 273.15


AttributeError: can't set attribute

In [57]:
t = temperature(C=0)
t._K = -50
print(f" La température vaut, en Kelvin : {t.Kelvin}")

 La température vaut, en Kelvin : -50


Si on veut pouvoir le modifier, il faut ajouter un setter avec le décorateur `@setter` :

In [43]:
def _CtoK(C):
    return C + 273.15

def _FtoK(F):
    return (F+459.67)*5/9

def _KtoC(K):
    return K - 273.15

def _KtoF(K):
    return 9/5*K-459.67


class temperature(object):
    """une classe pour la température"""
    def __init__(self, K=0, C=None, F=None):
        self._K = K
        if K is not None:
            self._K = K
        if C is not None:
            self._K = _CtoK(C)
        if F is not None:
            self._K = _FtoK(F)
            
    @property
    def Kelvin(self):
        return self._K
        
    @Kelvin.setter
    def Kelvin(self, K):
        if K>0 :
            self._K = K
        else :
            print("Temperature invalide")
            
    @property
    def Celsius(self):
        print("on appelle le getter")
        return _KtoC(self._K)
    
    @Celsius.setter
    def Celsius(self, C):
        print("on appelle le setter")
        self.Kevin = _CtoK(C)
            
    @property
    def Fahrenheit(self):
        return _KtoF(self._K)
    
    @Fahrenheit.setter
    def Fahrenheit(self, F):
        self.Kelvin = _FtoK(F)

    def __str__(self):
        return f"La température vaut K={self._K:.2f} (C={_KtoC(self._K):.2f}, F={_KtoF(self._K):.2f})"

In [44]:
T = temperature(C=0)
print(T)
print(T.Kelvin)
T.Fahrenheit = 50
print(T)
print(T.Celsius)

La température vaut K=273.15 (C=0.00, F=32.00)
273.15
La température vaut K=283.15 (C=10.00, F=50.00)
on appelle le getter
10.0


In [64]:
T = temperature(C=0)
print(f"la température en celsius est {T.Celsius} et en Kelvin est {T.Kelvin}")

on appelle le getter
la température en celsius est 0.0 et en Kelvin est 273.15


In [66]:
T.Celsius = 50
print(f"la température en celsius est {T.Celsius} et en Kelvin est {T.Kelvin}")

on appelle le setter
on appelle le getter
la température en celsius est 50.0 et en Kelvin est 323.15
